In [ ]:
# This code can be used to add a folder in the repository to the Python import
# path, irrespective of whether the notebook is being run in colab or Jupyter.
# (C) 2020 Abe Leite, Indiana University Bloomington
# This code block is released under MIT license. Feel free to make use of
# this code in any projects so long as you reproduce this text.

import os
import sys
import subprocess

repo_URL = 'https://github.com/ajleite/continuous-rl-exploration'
repo_name = repo_URL.split('/')[-1]
# top level of repository. --AL Mar 10 2022
code_folder = ''

try:
  repo_path = subprocess.check_output('git rev-parse --show-toplevel', shell=True).decode().strip()
except subprocess.CalledProcessError:
  os.system(f'git clone {repo_URL} --depth 1')
  repo_path = os.path.abspath(repo_name)

code_path = os.path.join(repo_path, code_folder)
sys.path.append(code_path)
print(f'Loading code from {code_path}')

In [ ]:
# don't forget the boilerplate --AL Mar 10 2022
!apt-get update
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install pyvirtualdisplay
!pip install piglet
!pip install gym
!pip install pybullet
!git clone https://github.com/benelot/pybullet-gym.git
cd /content/pybullet-gym/
!pip install -e .

In [ ]:
# we'll go to the repo path in order to work with saved models. --AL Mar 10 2022
os.chdir(code_path)

In [ ]:
import pickle

import numpy as np
import scipy.stats as st
import tensorflow as tf

import matplotlib
%matplotlib inline

import matplotlib.pyplot as plt

import gym

import pybullet_data
import pybulletgym

# 0 Tasks

For this assignment I make use of two tasks: the cart-pole task and the half-cheetah task. I make small modifications to each of them, which I describe below.

## 0.1 Cart-pole task

The cart-pole task is a classic control task introduced by Sutton and Barto in which a wheeled cart is mounted on a frictionless, finite-length beam. On top of the cart a pole is balanced. The agent's goal is to exert forces on the cart to prevent the pole from falling over without pushing the cart off of the edge of the beam.

For this task, I slightly modified the reward schedule to better reflect the structure of the task. This only affects the reward signals provided to the agent and not the episode returns shown in the graphs. The original cart-pole task does not distinguish between the case where the episode terminates naturally after 1000 timesteps and the case where the pole falls at timestep 1000. Additionally, by providing a positive reward for every timestep when the agent survives, it creates a slow-to-learn value function which will become, except in doomed trajectories, the constant-reward scale factor determined by the formula $\frac{1}{1-\gamma}$. Finally, the agent will be confused that the cumulative return for a timestep soon before the 1000-timestep cutoff will appear lower than the cumulative return at an earlier timestep even with an optimal policy. I rectify all of these issues by providing the agent with a 0 reward signal for every timestep that the pole does not fall and a -1 reward signal for any timestep that the pole does fall. The code that implements this version of the cart-pole problem is included below.

In [ ]:
class CartPoleTask:
    def __init__(self, rng):
        self.cartpole_env = gym.make("InvertedPendulumMuJoCoEnv-v0")
        self.cartpole_env.seed(int(rng.integers(2**63-1)))
        self.obs_shape = (4,)
        self.action_shape = (1,)
        self.ts = 0
    def reset(self):
        self.ts = 0
        return self.cartpole_env.reset()
    def step(self, action):
        o, r, t, _ = self.cartpole_env.step(action)
        self.ts += 1
        return o, -1. if t and self.ts != 1000 else 0., t, _
    def render(self):
        return self.cartpole_env.render()
    def get_return(self):
        return self.ts

## 0.2 Half-cheetah task

The half-cheetah task is a locomotion task in which an agent controls a two-dimensional cheetah-like body which is only stable when both legs are on the ground. As such, it can only move far distances by galloping rapidly, resulting in a deceptive-reward problem where the locally optimal behavior is always to take a single step forwards. The half-cheetah's body has 6 joints which are controlled by the agent and 17 proprioceptive position and velocity sensors which provide the agent with input. The agent is rewarded for forwards locomotion and penalized for energy cost.

I made only minimal modifications to the half-cheetah task. In order to reduce the dataset's bias towards states where the cheetah has fallen over, I early-terminate all episodes at 500 timesteps. This affects all episode return calculations. In cases where the agent is successfully galloping, this would roughly half the total return that the agent receives; in cases where the agent becomes stuck and does not move, it does not affect the total return. In cases where the agent is continually exerting torques but not moving successfully, it would roughly half the (negative) total return that the agent receives. The code that implements this version of the half-cheetah task is included below.

In [ ]:
class HalfCheetahTask:
    def __init__(self, rng, to_render=False):
        self.env = gym.make("HalfCheetahMuJoCoEnv-v0")
        if to_render:
            self.env.render('human')
        self.env.seed(int(rng.integers(2**63-1)))

        self.obs_shape = (17,)
        self.action_shape = (6,)
        self.cumulative_r = 0
        self.timestep = 0
    def reset(self):
        self.cumulative_r = 0
        self.timestep = 0
        return self.env.reset()
    def step(self, action):
        o, r, t, _ = self.env.step(action)
        self.timestep += 1
        self.cumulative_r += r
        return o, r, t or self.timestep == 500, _
    def render(self):
        return self.env.render('human')
    def get_return(self):
        return self.cumulative_r

## 0.3 Simulation model

The following cell contains boilerplate code for training an agent on an environment and saving the results. It greedily evaluates the policy every 50 training episodes in order to track how performance changes over time and preserve the best-performing weights.

In [ ]:
class Simulation:
    def __init__(self, agent, task, num_episodes, path=None):
        self.agent = agent
        self.task = task

        self.num_episodes = num_episodes

        self.training_episode_rewards = []
        self.eval_episode_rewards = []

        self.value_rmse = []
        self.mean_obj = []

        self.best_weights = None
        self.best_100_episode_return = None
        self.path = path

    def save_trace(self):
        if not self.path:
            return

        to_save = {'best_weights': self.best_weights, 'best_100_episode_return': self.best_100_episode_return,
            'training_episode_rewards': self.training_episode_rewards, 'eval_episode_rewards': self.eval_episode_rewards,
            'value_rmse': self.value_rmse, 'mean_obj': self.mean_obj}

        pickle.dump(to_save, open(self.path,'wb'))

    def evaluate(self, n, render=False):
        eval_rewards = []
        eval_lengths = []
        for i in range(100):
            s = self.task.reset()
            t = False

            while not t:
                a = self.agent.act(s, 0, temp=0)

                s2, r, t, _ = self.task.step(a)
                s = s2

                if render and i == 0:
                    self.task.render()

            # episode is over, record it
            eval_rewards.append(self.task.get_return())
            print((n, 'e', i, self.task.get_return()))

        self.eval_episode_rewards.append(eval_rewards)

        # maintain best stats
        if self.best_100_episode_return is None or np.mean(eval_rewards) > self.best_100_episode_return:
            self.best_100_episode_return = np.mean(eval_rewards)
            self.best_weights = self.agent.get_weights()
        print('cycle', n, 'mean eval:', np.mean(eval_rewards), 'best:', self.best_100_episode_return)
        self.save_trace()

    def run(self, render=False):
        timestep = 0

        for n in range(self.num_episodes):
            # 1. gather training trajectories
            training_rewards = []
            training_lengths = []

            for traj in range(self.agent.actor_count):
                s = self.task.reset()
                t = False

                while not t:
                    a = self.agent.act(s, traj)

                    s2, r, t, _ = self.task.step(a)
                    self.agent.store(traj, s, a, r, t)
                    s = s2

                    timestep += 1

                    if render and traj == 0:
                        self.task.render()

                # episode is over, record it
                training_rewards.append(self.task.get_return())
                print((n, 't', traj, self.task.get_return()))

            self.training_episode_rewards.append(training_rewards)

            # 2. train for 16 epochs
            value_rmse, mean_obj = self.agent.train(16)
            self.value_rmse.append(value_rmse)
            self.mean_obj.append(mean_obj)
            print(n, 'l', value_rmse, mean_obj)

            # 3. gather greedy evaluation trajectories every 50 training episodes
            if n % (50 / self.agent.actor_count) >= 1:
                continue

            self.evaluate(n, render)

        # everything is done!
        self.evaluate(self.num_episodes, render)

# 1 Network design

## 1.1 Probabilistic policy using mixture-of-Gaussians

I use a coordinate-wise mixture-of-Gaussians distribution as my stochastic policy; as such, my networks each output the paramaters for this distribution. One regret is that each action coordinate is independent of the others (each of the n action coordinates is determined using a univariate mixture-of-Gaussians, rather than the entire action being determined using an n-dimensional multivariate mixture-of-Gaussians). This prevents the multi-coordinate actions from being as coordinated as they would otherwise be.

Concretely, each mode of the multivariate Gaussian distribution has a weight, a mean, and a standard deviation. The shape of the policy network's output is thus `(n_action_coordinates, n_action_modes, 3)`, where the 3-dimension axis contains the weight, the mean, and the standard deviation. The weights are coerced into a discrete probability distribution using the softmax transformation, and the standard deviations are made non-negative using a sigmoid function. The sigmoid function's output is divided by the action's dimension in order to limit the total variance over all action coordinates. The means are clipped to the `[-1, 1]` range but are otherwise a direct linear transformation of the previous layer's output.

In order to maintain time-correlation for actions and allow for proper credit assignment, I recalculate the probability distribution coordinates for each mode selection and each Gaussian sample once every handful of frames according to a hyperparameter, `stoch_persistence`.

The code to select actions according to this probabilistic policy, calculate the log-probability of actions in the policy, and calculate the entropy of the policy (with certain limitations) is included below.

In [ ]:
class AdvantageAgent:
    def __init__(self, rng, actor_count, policy_network, value_network, t_max, discount_factor, stoch_persistence, entropy_weight):
        self.rng = rng

        self.actor_count = actor_count

        self.policy_network = policy_network
        self.value_network = value_network

        self.t_max = t_max
        self.discount_factor = discount_factor
        self.stoch_persistence = stoch_persistence

        self.beta_entropy_weight = entropy_weight
        self.minibatch_size = 128
        self.use_tqdm = False

        self.obs_shape = self.policy_network.obs_shape
        self.action_shape = self.policy_network.action_shape
        self.action_modes = self.policy_network.action_modes

        self.stoch_mode_t = self.rng.random((self.actor_count,) + self.action_shape)
        self.stoch_gauss_t = self.rng.random((self.actor_count,) + self.action_shape)
        # this is a prominent term in the Gaussian quantile formula, precomputed for efficiency
        self.stoch_gauss_inverf = np.sqrt(2) * scipy.special.erfinv(2 * self.stoch_gauss_t - 1)
        # this is a normalization factor to bias exploration down with high-dimensional actions
        self.stdev_norm = np.prod(self.action_shape)

        self.experience_buffer = [experience_store.NStepTDBuffer(self.obs_shape, self.action_shape, self.t_max, self.discount_factor) for _ in range(self.actor_count)]

    def get_weights(self):
        return self.policy_network.keras_network.get_weights(), self.value_network.keras_network.get_weights()

    def act(self, obs, actor=0, temp=1):
        if self.rng.random() > self.stoch_persistence:
            self.stoch_mode_t[actor] = self.rng.random(self.action_shape)
            self.stoch_gauss_t[actor] = self.rng.random(self.action_shape)
            self.stoch_gauss_inverf[actor] = np.sqrt(2) * scipy.special.erfinv(2 * self.stoch_gauss_t[actor] - 1)

        # shape: self.action_shape + (self.action_modes, 3), where the inner three values are the weight, mean, and standard deviation for each mode
        output = np.array(self.policy_network.apply(np.expand_dims(obs, axis=0))[0])
        weights = output[..., 0]
        pre_means = output[..., 1]
        pre_stdevs = output[..., 2]

        softmax_weights = np.exp(weights) / np.sum(np.exp(weights), axis=-1, keepdims=True)

        means = np.clip(pre_means, -1, 1)

        # this is the formula for the sigmoid function, with certain biases added
        stdevs = 1/(1 + np.exp(pre_stdevs))/self.stdev_norm+0.0001

        cum_weights = np.cumsum(softmax_weights, axis=-1)
        sel_mode = np.argmax(np.expand_dims(self.stoch_mode_t[actor], axis=-1) < cum_weights, axis=-1, keepdims=True)

        sel_means = np.squeeze(np.take_along_axis(means, sel_mode, axis=-1), axis=-1)
        sel_stdevs = np.squeeze(np.take_along_axis(stdevs, sel_mode, axis=-1), axis=-1)

        # this is the quantile formula of the Gaussian distribution
        sel_actions = sel_means + temp * sel_stdevs * self.stoch_gauss_inverf[actor]

        return sel_actions

    @tf.function
    def log_prob_actions(self, obss, actions):
        # take the weighted probability of each coordinate of each action over modes,
        #   then transform into log space and add over coordinates
        # maintains the batch axis

        # shape: (n,) + self.action_shape + (self.action_modes, 3)
        outputs = self.policy_network.apply(obss)
        weights = outputs[..., 0]
        pre_means = outputs[..., 1]
        pre_stdevs = outputs[..., 2]

        softmax_weights = tf.nn.softmax(weights, axis=-1)

        means = tf.clip_by_value(pre_means, -1, 1)

        stdevs = tf.nn.sigmoid(pre_stdevs)/self.stdev_norm+0.0001

        # want to get the PDF of each real action in each of the modes
        # this is the PDF formula of the Gaussian distribution
        mode_PDFs = tf.exp(-0.5 * ((means - tf.expand_dims(actions, axis=-1)) / stdevs)**2) / (stdevs * np.sqrt(2*np.pi))

        # take the weighted average over modes to get the probability of each coordinate of each action
        weighted_PDF = tf.reduce_sum(softmax_weights * mode_PDFs, axis=-1)+0.0001
        log_PDF = tf.math.log(weighted_PDF)

        # add in log space to get joint probability of all action coordinates, keeping the batch axis 0
        total_log_PDF = tf.reduce_sum(log_PDF, axis=range(1, len(log_PDF.shape)))

        return total_log_PDF

    @tf.function
    def action_entropies(self, obss):
        # Gets the entropy of the action distribution for each observation.
        # Unfortunately, I did not have time to find an analytic solution
        #   to the problem of finding the entropy of the mixture-of-Gaussians
        #   distribution. This is an open problem as described in
        #   https://isas.iar.kit.edu/pdf/MFI08_HuberBailey.pdf.
        # I have some novel techniques for describing the information theory
        #   of continuous random variables but I don't know if they will apply
        #   here.
        # Instead, I take the approach of overestimating entropy by assuming
        #   zero overlap between the component distributions.

        # shape: (n,) + self.action_shape + (self.action_modes, 3)
        outputs = self.policy_network.apply(obss)
        weights = outputs[..., 0]
        pre_means = outputs[..., 1]
        pre_stdevs = outputs[..., 2]
        stdevs = tf.nn.sigmoid(pre_stdevs)/self.stdev_norm+0.0001

        means = tf.clip_by_value(pre_means, -1, 1)

        # By my bag-of-tricks theorem (there is probably a better name for it somewhere),
        #   the entropy of a discrete mixture of RVs, selected according to some index RV
        #   X, is equal to to entropy of X plus the expected entropy of the selected RV.
        # Do not worry about the fact that I am mixing differential and discrete entropies.
        #   In my work unifying discrete and continuous information theory, I have shown
        #   that differential entropy is the finite deviation, in bits or nats, between the infinite
        #   entropies of your continuous random variable of interest and the unit uniform
        #   random variable of the same dimension.
        # So differential entropy and discrete entropy have the same units - nats - and all is well.

        mode_entropies = 0.5 * tf.math.log(2 * np.pi * stdevs**2) + 0.5

        # Because of clipping at +/-1, the above formula isn't quite right. Penalize means near +/- 1.
        mode_entropies -= means**4

        if self.action_modes > 1:
            softmax_weights = tf.nn.softmax(weights, axis=-1)
            weight_entropy_terms = -tf.math.log(softmax_weights)
            coordinate_entropies = tf.reduce_sum(tf.math.multiply_no_nan(weight_entropy_terms + mode_entropies, softmax_weights), axis=-1)
        else:
            coordinate_entropies = tf.reduce_sum(mode_entropies, axis=-1)

        # now we add the entropy of each coordinate since they are independent in this version.
        total_entropies = tf.reduce_sum(coordinate_entropies, axis=range(1, len(coordinate_entropies.shape)))

        return total_entropies

## 1.2 Network design (general)

In general, the networks' designs are quite simple; each consists of a number of feed-forward ReLU layers followed by an output layer whose shape is `(1,)` if the network is a value network or `(n_action_coordinates, n_action_modes, 3)` if the network is a policy network. The code necessary to support this is included below.

In [ ]:
def FFANN_factory(hidden_layer_sizes):
    def network_factory(obs_input):
        next_input = obs_input
        for hidden_layer, hidden_layer_size in enumerate(hidden_layer_sizes):
            next_input = tf.keras.layers.Dense(hidden_layer_size, activation='relu', kernel_initializer='random_normal', bias_initializer='random_normal')(next_input)
        return next_input

    return network_factory

class Network:
    def __init__(self, obs_shape, network_factory, learning_rate, is_policy, action_shape=None, action_modes=None):
        self.obs_shape = obs_shape
        self.network_factory = network_factory
        self.learning_rate = learning_rate
        self.is_policy = is_policy
        self.action_shape = action_shape
        self.action_modes = action_modes

        obs_input = tf.keras.layers.Input(shape=obs_shape)

        last_layer = network_factory(obs_input)

        if is_policy:
            output_shape = action_shape + (action_modes, 3)
            output_size = np.prod(output_shape)
        else:
            output_shape = (1,)
            output_size = 1

        flat_linear_output = tf.keras.layers.Dense(output_size)(last_layer)
        linear_output = tf.keras.layers.Reshape(output_shape)(flat_linear_output)

        self.keras_network = tf.keras.Model(obs_input, linear_output)

        self.optimizer = tf.keras.optimizers.Adam(learning_rate)

    @tf.function
    def apply(self, S):
        return self.keras_network(S)

    def copy_from(self, other, amount):
        for self_w, other_w in zip(self.keras_network.weights, other.keras_network.weights):
            self_w.assign(self_w*(1-amount) + other_w*amount)

    def copy(self):
        other = Network(self.obs_shape, self.network_factory, self.learning_rate, self.is_policy, self.action_shape, self.action_modes)
        other.copy_from(self, 1)
        return other

    def zero_like(self):
        other = Network(self.obs_shape, self.network_factory, self.learning_rate, self.is_policy, self.action_shape, self.action_modes)
        for other_w in other.keras_network.weights:
            other_w.assign(tf.zeros_like(other_w))
        return other

    def zero_self(self):
        for w in self.keras_network.weights:
            w.assign(tf.zeros_like(w))

## 1.3 Network design (cart-pole)

The cart-pole network design includes a 4-unit input layer from the environment, followed by 40- and 20-unit hidden ReLU layers, and finally either a 1-unit (value networks), 3-unit (policy network for REINFORCE), or 9-unit (policy network for A2C) linear output. The reason for the difference between REINFORCE's policy network and A2C's is that I used a unimodal policy distribution for REINFORCE, thinking it to be closer to the approach taken in the original paper. I use fully separate value and policy networks for program simplicity. These networks were optimized using Adam with a training rate of 0.0001. Although they are duplicated in Section 2, I have included the network specifications for both cart-pole experiments here.

In [ ]:
# REINFORCE
# policy_network = network.Network(task.obs_shape, network.FFANN_factory([40, 20]), 0.0001, True, task.action_shape, 1)
# value_network = network.Network(task.obs_shape, network.FFANN_factory([40, 20]), 0.0001, False, task.action_shape, 1)

# A2C
# policy_network = network.Network(task.obs_shape, network.FFANN_factory([40, 20]), 0.0001, True, task.action_shape, 3)
# value_network = network.Network(task.obs_shape, network.FFANN_factory([40, 20]), 0.0001, False, task.action_shape, 3)

## 1.4 Network design (half-cheetah)

The half-cheetah network design includes a 17-unit input layer from the environment, followed by 100- and 50-unit hidden ReLU layers, and finally either a 1-unit (value networks) or 18-unit (policy network) linear output. I am using larger hidden layers here to represent the numerous disentangled variables the network must maintain to successfully perform this task. Again, I use fully separate value and policy networks for program simplicity. These networks were optimized using Adam with a training rate of 0.00005. Although they are duplicated in Section 2, I have included the network specifications for the half-cheetah experiments here.

In [ ]:
# policy_network = network.Network(task.obs_shape, network.FFANN_factory([160, 80]), 0.0001, True, task.action_shape, 1)
# value_network = network.Network(task.obs_shape, network.FFANN_factory([160, 80]), 0.0001, False, task.action_shape, 1)

# 2 Training algorithms

## 2.1 Common features

REINFORCE and A2C are closely related reinforcement learning algorithms for probabilistic policies on continuous-action problems. Each of them is predicated on the idea of reinforcing actions that yield better-than-average results for the current policy. As such, each algorithm is highly dependent on on-policy data: without knowing the true average returns for each state it is impossible to know whether a particular action had positive advantage or not. The function representing the on-policy expected return of a state is called a baseline in REINFORCE and a value function in A2C.

In REINFORCE, the baseline is optional, though it is for practical purposes necessary whenever the mean return deviates from zero. In A2C, the value function is mandatory. In REINFORCE, the baseline is always trained on full rollout data, whereas in A2C it is often trained in the `n`-step TD manner where `n` is significantly less than the mean trajectory length.

In addition to network-level hyperparameters like training rate and policy-level hyperparameters like action persistence, one hyperparameter common to REINFORCE and A2C is the temporal discount factor, which is used in both the Monte-Carlo and the `n`-step TD calculations.

Code for training an `AdvantageAgent` that supports the key features of both REINFORCE and A2C is included below. Note that the agent's methods to act, calculate the log-probability of an action, and calculate the entropy of an action were all defined in Section 1.1. The code for the agent's experience store (which is cleared after every training cycle) is also included here.

In [ ]:
class NStepTDBuffer:
    def __init__(self, obs_shape, action_shape, t_max, discount_factor, buffer_size=10000):
        self.S_samples = np.zeros((buffer_size,)+obs_shape, dtype=np.float32)
        self.A_samples = np.zeros((buffer_size,)+action_shape, dtype=np.float32)
        self.R_samples = np.zeros((buffer_size,), dtype=np.float32)
        self.dt_samples = np.zeros((buffer_size,), dtype=np.int32)
        self.S2_samples = np.zeros((buffer_size,)+obs_shape, dtype=np.float32)

        self.t_max = t_max
        self.discount_factor = discount_factor

        # buffer_size represents the size of the buffer.
        # cur_index represents the next index that will be written.
        # filled represents whether the buffer has been filled at least once (can be sampled freely).
        self.buffer_size = buffer_size
        self.cur_index = 0
        self.filled = False

        self.episode_buffer = []

    def store_episode(self, observations, actions, rewards):
        ''' store_episode should be called at the end of an episode.

            observations, actions, and rewards should be numpy arrays
            whose shapes align along their first axis (timestep axis).

            - observations should be a float32 array whose subsequent axes match obs_shape.
            - actions should be a float32 array whose subsequent axes match action_shape.
            - rewards should be a float32 array with no subsequent axis,
              and it should represent the rewards for the transitions
              following the (observation, action) pairs in the corresponding indices. '''

        trajectory_length = rewards.shape[0]

        # discard unusable information ASAP
        if trajectory_length > self.buffer_size:
            print(f"WARNING: experience buffer overflow (trajectory length {trajectory_length}, buffer size {self.buffer_size}). Clipping beginning of trajectory.")
            observations = observations[-self.buffer_size:]
            actions = actions[-self.buffer_size:]
            rewards = rewards[-self.buffer_size:]
            trajectory_length = self.buffer_size

        t_max = self.t_max
        if t_max <= 0:
            t_max = trajectory_length

        # calculate trajectory rewards for each timestep
        # (ensure double precision for this calculation)
        traj_rewards = np.array(rewards, dtype=np.float64)
        for offset in range(1, min(trajectory_length, t_max)):
            traj_rewards[:-offset] += rewards[offset:] * self.discount_factor**offset
        traj_rewards = np.float32(traj_rewards)
        # print(traj_rewards)

        if trajectory_length <= t_max:
            dt = np.zeros(trajectory_length, dtype=np.int32)
            s2 = observations
        else:
            dt_incomplete = np.full(trajectory_length - t_max, t_max, dtype=np.int32)
            dt_complete = np.zeros(t_max, dtype=np.int32)
            dt = np.concatenate([dt_incomplete, dt_complete], axis=0)

            s2_incomplete = observations[:trajectory_length - t_max]
            s2_complete = np.zeros_like(observations[trajectory_length - t_max:])
            s2 = np.concatenate([s2_incomplete, s2_complete], axis=0)

        # store the trajectory in the buffer
        # (split based on whether we will wrap around the end of the buffer)
        will_loop = self.cur_index + trajectory_length >= self.buffer_size
        if will_loop:
            can_store = self.buffer_size - self.cur_index
            self.S_samples[self.cur_index:] = observations[:can_store]
            self.A_samples[self.cur_index:] = actions[:can_store]
            self.R_samples[self.cur_index:] = traj_rewards[:can_store]
            self.dt_samples[self.cur_index:] = dt[:can_store]
            self.S2_samples[self.cur_index:] = s2[:can_store]

            leftover = trajectory_length - can_store
            if leftover:
                self.S_samples[:leftover] = observations[can_store:]
                self.A_samples[:leftover] = actions[can_store:]
                self.R_samples[:leftover] = traj_rewards[can_store:]
                self.dt_samples[:leftover] = dt[can_store:]
                self.S2_samples[:leftover] = s2[can_store:]

            self.filled = True
            self.cur_index = (self.cur_index + trajectory_length) - self.buffer_size
        else:
            new_index = self.cur_index + trajectory_length

            self.S_samples[self.cur_index:new_index] = observations
            self.A_samples[self.cur_index:new_index] = actions
            self.R_samples[self.cur_index:new_index] = traj_rewards
            self.dt_samples[self.cur_index:new_index] = dt
            self.S2_samples[self.cur_index:new_index] = s2

            self.cur_index = new_index

    def store(self, obs, action, reward, terminal):
        ''' This is a convenience function to allow the MonteCarloBuffer to act more like the TD0Buffer.
            It matches the signature of TD0Bufer.store. '''

        self.episode_buffer.append((obs, action, reward))

        # This is an unusual idiom that allows one to effectively take the transpose of a Python list.
        # I often use it in RL contexts.
        if terminal:
            all_S, all_A, all_R = [np.array(all_samples) for all_samples in zip(*self.episode_buffer)]
            self.store_episode(all_S, all_A, all_R)
            self.episode_buffer = []

    def report(self):
        if self.filled:
            return (self.S_samples, self.A_samples, self.R_samples, self.dt_samples, self.S2_samples)
        else:
            return (self.S_samples[:self.cur_index], self.A_samples[:self.cur_index], self.R_samples[:self.cur_index],
                self.dt_samples[:self.cur_index], self.S2_samples[:self.cur_index])

    def clear(self):
        self.cur_index = 0
        self.filled = False

        self.episode_buffer = []

@tf.function
def AdvantageAgent_train_iter(self, S, A, R):
    cur_value = self.value_network.apply(S)[:,0]
    adv = R - cur_value

    log_prob_actions = self.log_prob_actions(S, A)
    action_entropies = self.action_entropies(S)
    obj = tf.reduce_sum(log_prob_actions * adv + self.beta_entropy_weight * action_entropies)

    obj_gradient = tf.gradients(-obj, self.policy_network.keras_network.weights)
    self.policy_network.optimizer.apply_gradients(zip(obj_gradient, self.policy_network.keras_network.weights))

    value_loss = tf.reduce_sum(adv ** 2)
    value_gradient = tf.gradients(value_loss, self.value_network.keras_network.weights)
    self.value_network.optimizer.apply_gradients(zip(value_gradient, self.value_network.keras_network.weights))

    return obj, value_loss

AdvantageAgent.train_iter = AdvantageAgent_train_iter

def AdvantageAgent_train(self, epochs=1):
    all_S = []
    all_A = []
    all_R = []
    # S2 and dt are used for the TD updates.
    # S2 is the state after all short-term reward has been received.
    # dt is the number of timesteps separating S from S2.
    # If dt is 0, then S2 is full of garbage values because the episode ended.
    all_dt = []
    all_S2 = []

    for buffer in self.experience_buffer:
        S, A, R, dt, S2 = buffer.report()
        buffer.clear()

        all_S.append(S)
        all_A.append(A)
        all_R.append(R)
        all_dt.append(dt)
        all_S2.append(S2)

    all_S = np.concatenate(all_S, axis=0)
    all_A = np.concatenate(all_A, axis=0)
    all_R = np.concatenate(all_R, axis=0)
    all_dt = np.concatenate(all_dt, axis=0)
    all_S2 = np.concatenate(all_S2, axis=0)

    terminal = (all_dt == 0)

    # use TD to estimate the total (short-term + long-term) expected reward
    if not np.all(terminal):
        all_R = np.where(terminal, all_R, all_R + self.discount_factor ** all_dt * np.array(self.value_network.apply(all_S2))[:,0])
        all_R = np.float32(all_R)
    del all_dt
    del all_S2

    total_training_samples = all_S.shape[0]
    num_minibatches = (total_training_samples * epochs) // self.minibatch_size
    last_minibatch_size = (total_training_samples * epochs) % self.minibatch_size

    total_obj = 0
    total_value_loss = 0

    if self.use_tqdm:
        import tqdm
        minibatches = tqdm.tqdm(range(num_minibatches))
    else:
        minibatches = range(num_minibatches)

    for _ in minibatches:
        sample_indices = self.rng.integers(total_training_samples, size=(self.minibatch_size))
        S = all_S[sample_indices]
        A = all_A[sample_indices]
        R = all_R[sample_indices]
        if S.size > 0:
            obj, value_loss = self.train_iter(S, A, R)
            total_obj += obj
            total_value_loss += value_loss

    if last_minibatch_size:
        sample_indices = self.rng.integers(total_training_samples, size=(last_minibatch_size))
        S = all_S[sample_indices]
        A = all_A[sample_indices]
        R = all_R[sample_indices]
        if S.size > 0:
            obj, value_loss = self.train_iter(S, A, R)
            total_obj += obj
            total_value_loss += value_loss

    value_rmse = np.sqrt(np.array(total_value_loss) / total_training_samples / epochs)
    mean_obj = np.array(total_obj) / total_training_samples / epochs

    return value_rmse, mean_obj

AdvantageAgent.train = AdvantageAgent_train

## 2.2 Unique features of REINFORCE

In my implementation, there are no unique features of REINFORCE compared with A2C. It is rather defined by the features it lacks, which will be discussed in the next section.

### 2.2.1 Training a policy for the cart-pole task using REINFORCE

In order to train my cart-pole policy using REINFORCE, I used a temporal discount factor of 0.95 (reward half-life: 13.5 timesteps, constant reward scaling factor: 20) and an action persistence of 0.965 (expected switch time: 20 timesteps). I trained for 4000 episodes (keeping in mind that episode length is variable based on current performance) and (as mentioned before) the network architectures were [4, 40, 20, 1] (value) and [4, 40, 20, 3] (policy) with a training rate of 0.0001. The full specification for the REINFORCE cart-pole simulation is included below.

In [ ]:
def test_REINFORCE_cartpole(seed):
    agent_rng = np.random.default_rng(seed)
    task_rng = np.random.default_rng(seed+234579672983459873)

    task = CartPoleTask(task_rng)

    path = f'out/REINFORCE-cartpole-{seed}.pickle'

    # expected time to switch action distribution is 20 timesteps
    policy_network = Network(task.obs_shape, network.FFANN_factory([40, 20]), 0.0001, True, task.action_shape, 1)
    value_network = Network(task.obs_shape, network.FFANN_factory([40, 20]), 0.0001, False, task.action_shape, 1)
    ag = AdvantageAgent(agent_rng, 1, policy_network, value_network, 0, 0.95, 0.965, 0)

    sim = Simulation(ag, task, 4000, path)
    sim.run(False)

### 2.2.2 Training a policy for the half-cheetah task using REINFORCE

In order to train my half-cheetah policy using REINFORCE, I used a temporal discount factor of 0.99 (reward half-life: 62.5 timesteps, constant reward scaling factor: 100) and an action persistence of 0.931 (expected switch time: 10 timesteps). I trained for 2500 episodes (capped at 500 timesteps each) and (as mentioned before) the network architectures were [17, 160, 80, 1] (value) and [17, 160, 180, 18] (policy) with a training rate of 0.00001. The full specification for the REINFORCE half-cheetah simulation is included below.

In [ ]:
def test_REINFORCE_cheetah(seed):
    agent_rng = np.random.default_rng(seed)
    task_rng = np.random.default_rng(seed+234579672983459873)

    task = tasks.HalfCheetahTask(task_rng)

    path = f'out/REINFORCE-cheetah-{seed}.pickle'

    # expected time to switch action distribution is 10 timesteps
    policy_network = network.Network(task.obs_shape, network.FFANN_factory([160, 80]), 0.00001, True, task.action_shape, 3)
    value_network = network.Network(task.obs_shape, network.FFANN_factory([160, 80]), 0.00001, False, task.action_shape, 3)
    ag = agent.AdvantageAgent(agent_rng, 1, policy_network, value_network, 0, 0.99, 0.931, 0)

    sim = simulation.Simulation(ag, task, 2500, path)
    sim.run(False)

## 2.3 Unique features of A2C

There are three key features that sets A2C apart from REINFORCE. First and foremost, it uses multiple actors to gather multiple trajectories using the same policy. Each actor will at the very least have its own action stochasticity, and in the original A2C implementation, different actors even had different exploration policies (which complicates the theoretical convergence properties of A2C by giving it a heterogeneous policy). This is special because, while each actor's trajectories will be internally correlated along both the actor's action stochasticity and the task's dynamics, the actors' trajectories will not be correlated with each other. This allows the correlated-samples problem to be solved without amassing gigantic datasets and slowing down the rate of policy iteration. (Note that my implementation does not take full advantage of this property since I only perform updates at the end of episodes.)

Second, and relatedly, A2C is able to use `n`-step TD bootstrapping while learning state values. While I have fully implemented this, I use Monte-Carlo value estimation to focus more closely on the other differences between my two learning algorithms.

Third, A2C avoids premature convergence by including the current policy's entropy over recently-seen states as a term in its objective function.

Each of these features was included in the above code for `AdvantageAgent`.

### 2.3.1 Training a policy for the cart-pole task using A2C

In order to train my cart-pole policy using A2C, I used a temporal discount factor of 0.95 (reward half-life: 13.5 timesteps, constant reward scaling factor: 20) and an action persistence of 0.965 (expected switch time: 20 timesteps). I included an entropy term weighted by 0.05 in my objective function. I used 25 actors and I trained for 160 episode batches (4000 episodes total). As mentioned before, the network architectures were \[4, 40, 20, 1\] (value) and \[4, 40, 20, 9\] (policy) with a training rate of 0.0001. The full specification for the A2C cart-pole simulation is included below.

In [ ]:
def test_A2C_cartpole(seed):
    agent_rng = np.random.default_rng(seed)
    task_rng = np.random.default_rng(seed+234579672983459873)

    task = tasks.CartPoleTask(task_rng)

    path = f'out/A2C-cartpole-{seed}.pickle'

    # expected time to switch action distribution is 20 timesteps
    policy_network = network.Network(task.obs_shape, network.FFANN_factory([40, 20]), 0.0001, True, task.action_shape, 3)
    value_network = network.Network(task.obs_shape, network.FFANN_factory([40, 20]), 0.0001, False, task.action_shape, 3)
    ag = agent.AdvantageAgent(agent_rng, 25, policy_network, value_network, 0, 0.95, 0.965, 0.05)

    sim = simulation.Simulation(ag, task, 160, path)
    sim.run(False)

### 2.3.2 Training a policy for the half-cheetah task using A2C

In order to train my half-cheetah policy using A2C, I used a temporal discount factor of 0.99 (reward half-life: 62.5 timesteps, constant reward scaling factor: 100) and an action persistence of 0.931 (expected switch time: 10 timesteps). I included an entropy term weighted by 0.05 in my objective function. I used 10 actors and trained for 250 episode cycles (2500 episodes total). As mentioned before, the network architectures were \[17, 160, 80, 1\] (value) and \[17, 160, 180, 18\] (policy) with a training rate of 0.00001. The full specification for the A2C half-cheetah simulation is included below.

In [ ]:
def test_A2C_cheetah(seed):
    agent_rng = np.random.default_rng(seed)
    task_rng = np.random.default_rng(seed+234579672983459873)

    task = tasks.HalfCheetahTask(task_rng)

    path = f'out/A2C-cheetah-{seed}.pickle'

    # expected time to switch action distribution is 20 timesteps
    policy_network = network.Network(task.obs_shape, network.FFANN_factory([160, 80]), 0.00001, True, task.action_shape, 3)
    value_network = network.Network(task.obs_shape, network.FFANN_factory([160, 80]), 0.00001, False, task.action_shape, 3)
    ag = agent.AdvantageAgent(agent_rng, 10, policy_network, value_network, 0, 0.99, 0.931, 0.05)

    sim = simulation.Simulation(ag, task, 250, path)
    sim.run(False)

## 2.4 Main simulations

Uncomment the line disabling the following cell and run it to reproduce the simulations for the main assignment. The results are all already saved in the repository

In [ ]:
enable_simulations_main = False

# enable_simulations_main = True

if enable_simulations_main:
    for i in range(20):
        test_REINFORCE_cartpole(i)
        test_A2C_cartpole(i)

    for i in range(20):
        test_REINFORCE_cheetah(i)
        test_A2C_cheetah(i)

# 3 Experiment results

## 3.0 Plotting code

The following code was used to report and plot my experiment results.

In [ ]:
def last_n_average(values, n):
    values = np.array(values)
    n = min(n, values.size)
    num = np.cumsum(values)
    den = np.cumsum(np.ones_like(values))
    prev_num = np.concatenate([np.zeros(n), num[:-n]], axis=0)
    prev_den = np.concatenate([np.zeros(n), den[:-n]], axis=0)
    return (num-prev_num)/(den-prev_den)

def load_records(condition, seeds):
    records = []
    for s in seeds:
        records.append(pickle.load(open(f'out/{condition}-{s}.pickle', 'rb')))
    return records

def plot_training_curves(records, suptitle='Training progress', out_fn=None):
    training_returns = [[np.mean(r) for r in record['training_episode_rewards']] for record in records]
    evaluation_returns = [[np.mean(r) for r in record['eval_episode_rewards']] for record in records]
    value_rmse = [record['value_rmse'] for record in records]
    mean_obj = [record['mean_obj'] for record in records]
    training_cycle_length = len(records[0]['training_episode_rewards'][0])
    training_cycle_count = len(records[0]['training_episode_rewards'])
    training_episode_count = training_cycle_length * training_cycle_count

    training_episode_numbers = np.arange(0, training_episode_count, training_cycle_length) + training_cycle_length / 2

    eval_cycle_length = 50

    eval_episode_numbers = np.concatenate([np.arange(0, training_episode_count, eval_cycle_length), [training_episode_count]], axis=0)

    plt.figure()
    for i, title, x_axis, runs in zip([1,2,3,4], ['On-policy return', 'Greedy return', 'Value loss (RMSE)', 'Mean objective'], [training_episode_numbers, eval_episode_numbers, training_episode_numbers, training_episode_numbers], [training_returns, evaluation_returns, value_rmse, mean_obj]):
        if not (runs is training_returns and training_cycle_length != 1 or runs is evaluation_returns):
            runs = [last_n_average(run, 100) for run in runs]
            # title = title+' (running mean)'

        plt.subplot(2,2,i)
        plt.title(title)

        if len(runs) > 1:
            mean_run = np.mean(runs, axis=0)

            run_l, run_h = st.t.interval(0.95, len(runs)-1, loc=mean_run, scale=st.sem(runs, axis=0))

            plt.fill_between(x_axis, run_l, run_h, color='black', alpha=0.25)
            plt.plot(x_axis, run_l, color='black', lw=0.5, ls='--')
            plt.plot(x_axis, run_h, color='black', lw=0.5, ls='--', label='95% c.i.')

        label = (i == 2)
        for run in runs:
            if len(runs) == 1:
                plt.plot(x_axis, run, label='indiv. run')
            elif label:
                plt.plot(x_axis, run, alpha=0.25, label='indiv. run')
            else:
                plt.plot(x_axis, run, alpha=0.25)
            label = False

        if len(runs) > 1:
            plt.plot(x_axis, mean_run, color='red', label='mean run')

        if i == 2:
            plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')

        if i > 2:
            plt.xlabel('Episode')

    plt.suptitle(suptitle)
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.78, hspace=0.3, wspace=0.25)

    if not out_fn is None:
        plt.savefig(out_fn)
        plt.close()

def plot_performance(REINFORCE_records, A2C_records, task_name, out_fn=None):
    REINFORCE_perfs = [record['best_100_episode_return'] for record in REINFORCE_records]
    A2C_perfs = [record['best_100_episode_return'] for record in A2C_records]

    print(f"REINFORCE on {task_name}: {np.mean(REINFORCE_perfs)} +/- {np.std(REINFORCE_perfs)}")
    print(f"A2C on {task_name}: {np.mean(A2C_perfs)} +/- {np.std(A2C_perfs)}")

    plt.figure()
    plt.title(f'Per-run best performance on {task_name} task')
    plt.violinplot([REINFORCE_perfs, A2C_perfs], showmeans=True)
    plt.gca().xaxis.set_ticks([1,2],['REINFORCE','A2C'])
    plt.ylabel('100-episode mean return')

    if not out_fn is None:
        plt.savefig(out_fn)
        plt.close()

## 3.1.1 Results for REINFORCE on the cart-pole task

In [ ]:
plot_training_curves(load_records("REINFORCE-cartpole"))

# 5 Learning the half-cheetah from images

## 5.1 Network design

Based on my experience with the Pong assignment, I knew that I would not be able to train my full network end-to-end on this problem in a reasonable amount of time. Instead, I turned to a modular approach using a variational autoencoder as a vision module. For this bonus, I used my preferred variational autoencoder implementation, which I wrote two years ago and adapted for this task. This implementation is based on the VAE used in Schmidhuber and Ha's World Models paper. I use the latent representation of the visual stimuli from the half-cheetah task as the input to my agent.

The main code for my variational autoencoder is included below.

In [ ]:
# closely based on my preferred VAE implementation for my own research, which is in turn
# loosely based on https://www.tensorflow.org/tutorials/generative/cvae

# loss functions and encoder/decoder architecture borrowed from Ha's repository at
# https://github.com/hardmaru/WorldModelsExperiments/blob/master/carracing/vae/vae.py
# in order to reproduce the Schmidhuber and Ha results.

def log_normal_pdf(x, mean, std_dev):
    var = std_dev*std_dev
    return -.5 * ((x - mean) ** 2. / var + tf.math.log(var*2*np.pi))

class VAE:
    def __init__(self, inference_conv_layers, generator_deconv_layers, image_shape, latent_dim):
        self.image_shape = image_shape
        self.latent_dim = latent_dim

        self.inference_layers = [tf.keras.layers.InputLayer(input_shape=image_shape)] + \
            inference_conv_layers + [tf.keras.layers.Flatten(), tf.keras.layers.Dense(2*latent_dim)]
        self.inference_net = tf.keras.Sequential(self.inference_layers)
        self.generator_layers = [tf.keras.layers.InputLayer(input_shape=(latent_dim,))] + generator_deconv_layers
        self.generator_net = tf.keras.Sequential(self.generator_layers)

        self.optimizer = tf.keras.optimizers.Adam(0.0005)

    @tf.function
    def get_mean(self, sample):
        if sample.dtype == tf.uint8:
            sample = tf.cast(sample, 'float32')/255
        latent_info = self.inference_net(sample)
        mean, log_var = tf.split(latent_info, [self.latent_dim, self.latent_dim], -1)
        return mean

    @tf.function
    def sample_latent(self, sample):
        if sample.dtype == tf.uint8:
            sample = tf.cast(sample, 'float32')/255
        if len(sample.shape) == 3:
            unbatched = True
            sample = tf.expand_dims(sample, axis=0)
        else:
            unbatched = False

        latent_info = self.inference_net(sample)
        mean, log_var = tf.split(latent_info, [self.latent_dim, self.latent_dim], -1)

        std_dev = tf.exp(log_var / 2.0)

        sampled_t = tf.random.normal(shape=std_dev.shape)
        sampled_z = mean + std_dev*sampled_t

        if unbatched:
            return sampled_z[0]
        else:
            return sampled_z

    @tf.function
    def train_vae(self, sample):
        ''' Sampling-based approach that gets means and standard deviations
            from inference(sample), then per batch item uses a single std normal
            sample to pick a vector for the generator to use.
            (With small batches and no momentum, this can result in unexpected
            gradients for the std deviation term. Fortunately, we are using Adam here.) '''
        if sample.dtype == tf.uint8:
            sample = tf.cast(sample, 'float32')/255
        latent_info = self.inference_net(sample)
        mean, log_var = tf.split(latent_info, [self.latent_dim, self.latent_dim], -1)

        std_dev = tf.exp(log_var / 2.0)

        sampled_t = tf.random.normal(shape=std_dev.shape)
        sampled_z = mean + std_dev*sampled_t

        new = self.generator_net(sampled_z)

        # lifted from David Ha's github
        # Ha replaces the traditional probability-distribution loss with two things:
        # the error in reconstructing the sample
        reconstruction_loss = tf.reduce_sum((sample-new)**2) / sample.shape[0]
        # and the KL-divergence of the z-distribution from the standard normal distribution
        kl_loss = - 0.5 * tf.reduce_sum(1 + log_var - mean**2 - tf.exp(log_var)) / sample.shape[0]

        loss = reconstruction_loss + kl_loss

        cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=new, labels=sample) # is this what we want?
        logpsample_z = -tf.reduce_sum(cross_ent, axis=[1, 2, 3])
        # the unconditioned latent distribution is assumed to be standard normal:
        logpz = tf.reduce_sum(log_normal_pdf(sampled_z, 0., 1.), axis=[1])
        logqz_sample = tf.reduce_sum(log_normal_pdf(sampled_z, mean, std_dev), axis=[1])
        loss = -tf.reduce_mean(logpsample_z + logpz - logqz_sample)

        vars = self.inference_net.trainable_variables + self.generator_net.trainable_variables
        grads = tf.gradients(loss, vars)
        self.optimizer.apply_gradients(zip(grads, vars))

        return loss

The VAE includes an encoder network and a decoder network. The encoder network consists of four convolution layers, each of which applies 2x2 filters with a stride of 2x2 followed by a ReLU nonlinearity. There are no pooling layers. The convolution layers have 32, 64, 128, and 256 filters, respectively. Following the convolution layers, the feature maps are flattened and fed through a fully-connected layer with a 16-unit linear output that parameterizes the normal distribution of latent-space embeddings of the image. (The first 8 units are the means and the second 8 are the log-variances for the distribution.) Keep in mind that, since a VAE is a probabilistic model for what sort of latent-space representation *might* have yielded the observed surface sample, there is randomness involved in the `sample_latent` method.

The decoder network consists of a single fully-connected ReLU layer that maps the 8-unit latent representation onto a 1x1x1024 representation, followed by four convolution transpose layers, which consist of 128 5x5 filters applied with a stride of 2, 64 5x5 filters applied with a stride of 2, 32 7x7 filters applied with a stride of 2, and 3 6x6 filters applied with a stride of 3x3. This full architecture is identical to the one used in Schmidhuber and Ha's "World Models" paper from a few years ago, except that I use a much smaller 8-unit latent representation instead of their 32-unit representation.

The code to generate a VAE with this specification and to train it on the half-cheetah data follows.

In [ ]:
# pulled from Schmidhuber & Ha "World Models"
def make_default_vae(image_shape=(96, 96, 3), latent_dim=8):
    sample_inference_conv_layers = [
        tf.keras.layers.Conv2D(filters=32, kernel_size=2, strides=(2, 2), activation='relu'),
        tf.keras.layers.Conv2D(filters=64, kernel_size=2, strides=(2, 2), activation='relu'),
        tf.keras.layers.Conv2D(filters=128, kernel_size=2, strides=(2, 2), activation='relu'),
        tf.keras.layers.Conv2D(filters=256, kernel_size=2, strides=(2, 2), activation='relu')]

    sample_generator_deconv_layers = [tf.keras.layers.Dense(units=1*1*1024, activation=tf.nn.relu),
        tf.keras.layers.Reshape(target_shape=(1, 1, 1024)),
        tf.keras.layers.Conv2DTranspose(filters=128, kernel_size=5, strides=(2, 2), padding="VALID", activation='relu'),
        tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=5, strides=(2, 2), padding="VALID", activation='relu'),
        tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=7, strides=(2, 2), padding="VALID", activation='relu'),
        tf.keras.layers.Conv2DTranspose(filters=3, kernel_size=6, strides=(3, 3), padding="VALID")]

    return VAE(sample_inference_conv_layers, sample_generator_deconv_layers, image_shape, latent_dim)

## 5.2 Training

### 5.2.1 VAE training regime

The goal here is to use random rollouts to obtain a wide variety of images for the half-cheetah task, which should ideally represent a good portion of the distribution that will be encountered during reinforcement learning training. For a previous course project, I studied a possible approach to iteratively train a VAE and the RL agent it acts as a vision module for, but I do not employ that here.

Code to gather and save these images follows. Note that the resulting dataset is 8 GB in size.

In [ ]:
class RandomAgent:
    def __init__(self, rng, actor_count, action_shape, stoch_persistence):
        self.rng = rng

        self.actor_count = actor_count

        self.stoch_persistence = stoch_persistence

        self.action_shape = action_shape

        self.cur_action = self.rng.normal(size=(self.actor_count,) + self.action_shape)

    def act(self, obs, actor=0, temp=1):
        if self.rng.random() > self.stoch_persistence:
            self.cur_action[actor] = self.rng.normal(size=self.action_shape)

        return self.cur_action[actor]

    def store(self, actor, obs, action, reward, terminal):
        pass

    def train(self, epochs=1):
        return np.array(0.), np.array(0.)

    def get_weights(self):
        return None

class HalfCheetahVAETask:
    def __init__(self, rng, gen_images=False):
        import vae
        self.vae = vae.make_default_vae(latent_dim=8)

        self.rng = rng

        self.env = gym.make("HalfCheetahMuJoCoEnv-v0")
        self.env.seed(int(rng.integers(2**63-1)))
        self.env.env._render_width = 96
        self.env.env._render_height = 96

        self.obs_shape = (self.vae.latent_dim,)
        self.action_shape = (6,)
        self.cumulative_r = 0

        self.gen_images = gen_images

        if self.gen_images:
            self.im_buffer = np.zeros((1024, 96, 96, 3))
            self.im_buffer_i = 0
        else:
            self.vae.inference_net.load_weights('out/cheetah_inf.tfdat')
            self.vae.generator_net.load_weights('out/cheetah_gen.tfdat')

    def reset(self):
        self.cumulative_r = 0
        self.env.reset()

        if self.gen_images:
            obs = np.zeros(self.obs_shape)
        else:
            vis_obs = self.env.render('rgb_array')
            obs = self.vae.sample_latent(vis_obs)

        return obs

    def step(self, action):
        _, r, t, _ = self.env.step(action)

        if (not self.gen_images) or self.rng.random() < 0.1:
            vis_obs = self.env.render('rgb_array')

            if self.gen_images:
                self.im_buffer[self.im_buffer_i] = vis_obs
                self.im_buffer_i += 1
                if self.im_buffer_i == self.im_buffer.shape[0]:
                    self.im_buffer = np.concatenate([self.im_buffer, np.zeros((1024, 96, 96, 3))], axis=0)

        if self.gen_images:
            obs = np.zeros(self.obs_shape)
        else:
            obs = self.vae.sample_latent(vis_obs)

        self.cumulative_r += r
        return obs, r, t, _

    def render(self):
        return self.env.render()
   
    def get_return(self):
        return self.cumulative_r

    def get_samples(self):
        return self.im_buffer[:self.im_buffer_i]

def gen_random_cheetah_rollouts(seed):
    agent_rng = np.random.default_rng(seed)
    task_rng = np.random.default_rng(seed+234579672983459873)

    task = tasks.HalfCheetahVAETask(task_rng, no_state=True)

    # expected time to switch action distribution is 10 timesteps
    ag = agent.RandomAgent(agent_rng, 1, task.action_shape, 0.931)

    sim = simulation.Simulation(ag, task, 100)
    sim.run(True)

    np.save('out/half_cheetah_images.npy', task.get_samples())

Following this, we will train the VAE on this dataset for 8 epochs, When training and especially when fine-tuning VAEs, I like to understand how representations are changing over time. Thus, my training code keeps track of and plots the cosine similarity of a sample from the training set as well as the overall loss terms.

In [ ]:
def train_vae(vae, training_dataset, distance_sampler):
    losses = []
    distances = []
    first_mean = vae.get_mean(distance_sampler)
    for epoch in range(8):
        total = 0
        count = 0
        for sample in training_dataset:
            loss = vae.train_vae(sample)
            losses.append(loss.numpy())
            current_mean = vae.get_mean(distance_sampler)
            cos = -tf.reduce_mean(tf.keras.losses.cosine_similarity(first_mean, current_mean))
            distances.append(cos.numpy())
            total += loss
            count += 1
            if count % 50 == 0:
                print(count, loss)
        print('Epoch', epoch, total/count)
    plt.plot(losses, label='VAE Loss')
    plt.xlabel('Training Steps')
    plt.ylabel('VAE Loss')
    plt.show()
    plt.plot(distances, label='Sample Embedding Similarity')
    plt.xlabel('Training Steps')
    plt.ylabel('Sample Embedding Similarity')
    plt.show()

def train_cheetah_VAE():
    states = np.load('out/half_cheetah_images.npy')
    np.random.seed(15)
    np.random.shuffle(states)
    states = np.uint8(states)

    minibatch = 256

    training_dataset = tf.data.Dataset.from_tensor_slices(states).batch(minibatch)

    vae = make_default_vae(latent_dim=8)
    train_vae(vae, training_dataset, states[:10])

    vae.inference_net.save_weights('out/cheetah_inf.tfdat')
    vae.generator_net.save_weights('out/cheetah_gen.tfdat')

### 5.2.2 RL training regimes

In both cases, I use the VAE as a vision module and treat it effectively as a preprocessor, expecting the RL algorithms to operate exactly they as they did with the half-cheetah's proprioceptive inputs, except for the fact that the VAE's latent space has 8 dimensions and the original observation space has 17. The code to perform this is included above in the defintion of `HalfCheetahVAETask`.

#### 5.2.2.1 Training a policy for the half-cheetah VAE task using REINFORCE

My regime almost precisely follows the one described in section 2.2.2. In order to train my half-cheetah policy using REINFORCE with a VAE input, I used a temporal discount factor of 0.99 (reward half-life: 62.5 timesteps, constant reward scaling factor: 100) and an action persistence of 0.931 (expected switch time: 10 timesteps). I trained for 2500 episodes (capped at 500 timesteps each) and (as mentioned before) the network architectures were [8, 160, 80, 1] (value) and [8, 160, 180, 18] (policy) with a training rate of 0.00001. The full specification for the REINFORCE half-cheetah VAE simulation is included below.

In [ ]:
def test_REINFORCE_cheetah_VAE(seed):
    agent_rng = np.random.default_rng(seed)
    task_rng = np.random.default_rng(seed+234579672983459873)

    task = tasks.HalfCheetahVAETask(task_rng)

    path = f'out/REINFORCE-cheetahVAE-{seed}.pickle'

    # expected time to switch action distribution is 10 timesteps
    policy_network = network.Network(task.obs_shape, network.FFANN_factory([160, 80]), 0.00001, True, task.action_shape, 3)
    value_network = network.Network(task.obs_shape, network.FFANN_factory([160, 80]), 0.00001, False, task.action_shape, 3)
    ag = agent.AdvantageAgent(agent_rng, 1, policy_network, value_network, 0, 0.99, 0.931, 0)

    sim = simulation.Simulation(ag, task, 2500, path)
    sim.run(False)

#### 5.2.2.2 Training a policy for the half-cheetah VAE task using A2C

My regime almost precisely follows the one described in section 2.3.2. In order to train my half-cheetah policy using A2C with a VAE input, I used a temporal discount factor of 0.99 (reward half-life: 62.5 timesteps, constant reward scaling factor: 100) and an action persistence of 0.931 (expected switch time: 10 timesteps). I included an entropy term weighted by 0.05 in my objective function. I used 10 actors and trained for 250 episode cycles (2500 episodes total). As mentioned before, the network architectures were \[8, 160, 80, 1\] (value) and \[8, 160, 180, 18\] (policy) with a training rate of 0.00001. The full specification for the A2C half-cheetah VAE simulation is included below.

In [ ]:
def test_A2C_cheetah_VAE(seed):
    agent_rng = np.random.default_rng(seed)
    task_rng = np.random.default_rng(seed+234579672983459873)

    task = tasks.HalfCheetahVAETask(task_rng)

    path = f'out/A2C-cheetahVAE-{seed}.pickle'

    # expected time to switch action distribution is 20 timesteps
    policy_network = network.Network(task.obs_shape, network.FFANN_factory([160, 80]), 0.00001, True, task.action_shape, 3)
    value_network = network.Network(task.obs_shape, network.FFANN_factory([160, 80]), 0.00001, False, task.action_shape, 3)
    ag = agent.AdvantageAgent(agent_rng, 10, policy_network, value_network, 0, 0.99, 0.931, 0.05)

    sim = simulation.Simulation(ag, task, 250, path)
    sim.run(False)

### 5.2.3 Main VAE simulations

Enable the following cell to generate new random rollouts for the half-cheetah task and train the VAE. Again, this will consume 8 GB of disk space.

In [ ]:
enable_vae_training = False

# enable_vae_training = True

if enable_vae_training:
    gen_random_cheetah_rollouts(0)
    
    train_cheetah_VAE()

Enable and execute the following cell to repeat the experiments applying the two RL algorithms using a VAE to the task.

In [ ]:
enable_simulations_vae = False

# enable_simulations_vae = True

if enable_simulations_vae:
    for i in range(5):
        test_REINFORCE_cheetah_VAE(i)
        test_A2C_cheetah_VAE(i)